In [1]:
using TensorDecomposition, Symbolics, LinearAlgebra, InvertedIndices

┌ Info: Precompiling TensorDecomposition [51265e24-c02d-4141-bbd6-b01e1585b829]
└ @ Base loading.jl:1423


In [2]:
function rlim(n, d)
    if iseven(d)
         if n == 2
             return binomial(Int(n+d/2), n) - n
         else
             return binomial(Int(n+d/2), n) - n - 1
         end
     else 
         return binomial(Int(n+(d-1)/2), n)
     end
 end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end;

function alpha_iterator(::Val{N}, s, t=()) where {N}
    N <= 1 && return ((s, t...),) # Iterator over a single Tuple
    Iterators.flatten(alpha_iterator(Val(N-1), s-i, (i, t...)) for i in 0:s)
end

function monomialOrder(a, b)
    if a[1] > b[1]
        return true
    elseif a[1] < b[1]
        return false
    else 
        if maximum(a[2:end]) < maximum(b[2:end])
            return true
        elseif maximum(a[2:end]) > maximum(b[2:end])
            return false
        else
            return !isless(a[2:end], b[2:end])
        end 
    end
end

function convertIndices(x)
    d = sum(x)
    y = zeros(Int, d)
    c = 1
    for i=1:length(x)
        pow = x[i]
        y[c:c+pow-1] .= i
        c += pow
    end 
    return y .- 1
end;

function e(j, n)
    e = zeros(n)
    e[j] = 1
    return e
end;

function multBasis(basis, j, n)
    ej = e(j, n)
    ej[1] = -1
    return [Tuple(Int(y) for y in [x for x in b] + ej) for b in basis]
end

multMonomial(b, ej) = Tuple(Int(y) for y in [x for x in b] + ej);

### Real tensor

In [345]:
n = 9
d = 4;

In [347]:
r = rlim(n, d)

45

In [348]:
T, A_, L = randomRankedTensor(n+1, d, r, real=true);
T /= sqrt(n+1);

In [349]:
Thank = hankMat2(T);

In [350]:
# Thank

In [351]:
monomials = sort(collect(alpha_iterator(Val(n+1), Int(d))), lt=monomialOrder);
D = Dict()
for (i, mon) in enumerate(monomials)
   D[mon] = i 
end

basis = monomials[1:r];

In [352]:
first_r = [D[b] for b in basis];

In [353]:
H0 = Thank[first_r, first_r];

In [354]:
H0_adj = cofactor(Symbolics.value.(H0));

In [355]:
Hs = []
for i=2:n+1
    col_inds = [D[b] for b in multBasis(basis, i, n+1)]
    push!(Hs, Thank[first_r, col_inds])
end

In [356]:
vars = unique(reduce(vcat, ([reduce(vcat, Symbolics.get_variables.(H)) for H in Hs])));

In [357]:
monomialDegree(alpha) = sum(alpha[2:end]);

In [358]:
degree_d2 = []
degree_d2_1 = []

for b in basis
   if monomialDegree(b) == d/2-1
        push!(degree_d2_1, b)
    elseif monomialDegree(b) == d/2
        push!(degree_d2, b)
    end
end

In [359]:
linearEquations = []
for i=2:n+1
    ei = e(i, n+1)
    ei[1] = -1
    for j=i+1:n+1
        ej = e(j, n+1)
        ej[1] = -1
        
        for beta in degree_d2_1
            beta_i = multMonomial(beta, ei)
            beta_j = multMonomial(beta, ej)
            if !(beta_i in basis && beta_j in basis)
                for alpha in degree_d2
                    x1 = Thank[first_r, D[multMonomial(alpha, ei)]]
                    y1 = Thank[first_r, D[beta_j]]
                
                    x2 = Thank[first_r, D[beta_i]]
                    y2 = Thank[first_r, D[multMonomial(alpha, ej)]]
                    eq = transpose(x1)*H0_adj*y1-transpose(x2)*H0_adj*y2
                    push!(linearEquations, Symbolics.expand(eq))
                end
            end
        end
    end
end

In [ ]:
A = []
b = []
for eq in linearEquations
    push!(A, [Symbolics.coeff(eq, h_) for h_ in vars])
    push!(b, Symbolics.coeff(eq))
end
A = reduce(hcat,A)';
# A = map!(x -> isapprox(x, 0, atol=1e-10) ? 0 : x, A, A);
# A = round.(A, digits=10);
# A = unique(A, dims=1);

In [ ]:
vars

In [ ]:
svdvals(A)